In [1]:
!pip install wandb

In [2]:
import wandb
key = input('Enter your API:')
wandb.login(key=key)


Enter your API: 580e769ee2f34eafdded556ce52aaf31c265ad3b


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ma23m011 (ma23m011-iit-madras). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# 1. Data Loading and Transform
# -----------------------------
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to fixed size
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
])

train_data = ImageFolder(root='/kaggle/input/inutaral/inaturalist_12K/train', transform=transform)
test_data = ImageFolder(root='/kaggle/input/inutaral/inaturalist_12K/val', transform=transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)




class CustomCNN(nn.Module):
    def __init__(self, input_channels=3, num_filters=None, kernel_size=3, 
                 activation='relu', dense_neurons=128, num_classes=10, 
                 dropout_rate=0.3, batch_norm=True, num_conv_layers=5, input_size=128):
        super(CustomCNN, self).__init__()
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout_rate = dropout_rate

        # Handle num_filters as int or list
        if num_filters is None:
            num_filters = [32, 64, 128, 256, 512]
        elif isinstance(num_filters, int):
            num_filters = [num_filters] * num_conv_layers
        elif isinstance(num_filters, list):
            assert len(num_filters) >= num_conv_layers

        # Handle kernel_size as int or list
        if isinstance(kernel_size, int):
            kernel_size = [kernel_size] * num_conv_layers
        elif isinstance(kernel_size, list):
            assert len(kernel_size) >= num_conv_layers
        else:
            raise TypeError("kernel_size must be a list or an integer.")

        # Handle dense_neurons as int or list
        if isinstance(dense_neurons, int):
            dense_neurons = [dense_neurons]
        elif isinstance(dense_neurons, list):
            assert all(isinstance(x, int) for x in dense_neurons)
        else:
            raise TypeError("dense_neurons must be a list or an integer.")

        # Convolutional layers
        layers = []
        in_channels = input_channels
        for i in range(num_conv_layers):
            out_channels = num_filters[i]
            k_size = kernel_size[i]
            layers.append(self.create_conv_block(in_channels, out_channels, k_size))
            in_channels = out_channels
        self.conv_layers = nn.Sequential(*layers)

        # Dynamically calculate flattened size
        with torch.no_grad():
            dummy_input = torch.zeros(1, input_channels, input_size, input_size)
            dummy_output = self.conv_layers(dummy_input)
            self.flattened_size = dummy_output.view(1, -1).size(1)

        # Dense layers
        self.flatten = nn.Flatten()
        fc_layers = []
        in_features = self.flattened_size
        for out_features in dense_neurons:
            fc_layers.append(nn.Linear(in_features, out_features))
            fc_layers.append(self.get_activation())
            fc_layers.append(nn.Dropout(self.dropout_rate))
            in_features = out_features
        fc_layers.append(nn.Linear(in_features, num_classes))  # Final output layer
        self.fc_layers = nn.Sequential(*fc_layers)

    def create_conv_block(self, in_channels, out_channels, kernel_size):
        padding = kernel_size // 2
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding)]
        if self.batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(self.get_activation())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        layers.append(nn.Dropout(self.dropout_rate))
        return nn.Sequential(*layers)

    def get_activation(self):
        activations = {
            'relu': nn.ReLU(),
            'gelu': nn.GELU(),
            'silu': nn.SiLU(),
            'mish': nn.Mish()
        }
        return activations.get(self.activation, nn.ReLU())

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.fc_layers(x)
        return x

num_classes = len(train_data.classes)
model = CustomCNN(
    input_channels=3,
    input_size=128,
    num_classes=num_classes,
    num_conv_layers=5,
    num_filters=[32, 64, 128, 256, 512,1024],
    kernel_size=[3, 5, 3, 5, 1],  # ← Different kernel sizes for each conv layer
    dense_neurons=[512, 256, 64]
).to(device)

In [8]:
def train_model(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Create CNN model with hyperparameters
        model = CustomCNN(
            input_channels=3,
            num_filters=config.num_filters,
            activation=config.activation,
            dense_neurons=128,
            num_classes=10,
            dropout_rate=config.dropout_rate,
            batch_norm=config.batch_norm
        ).to(device)

        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Training loop
        for epoch in range(10):  # 10 epochs for demonstration
            model.train()
            running_loss = 0.0
            correct, total = 0, 0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            train_acc = 100 * correct / total
            val_acc, val_loss = evaluate_model(model, val_loader, criterion)

            # Log metrics
            wandb.log({'train_loss': running_loss / len(train_loader), 'train_accuracy': train_acc,
                       'val_loss': val_loss, 'val_accuracy': val_acc})



In [9]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    correct, total, running_loss = 0, 0, 0.0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = running_loss / len(data_loader)
    return accuracy, avg_loss

In [10]:
import wandb

sweep_config = {
    'method': 'random',  # or 'grid' / 'bayes'
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'num_filters': {'values': [[16, 32, 64, 128, 256], [64, 64, 64, 64, 64], [256, 128, 64, 32, 16], [32, 32, 32,32,32], [16, 32, 64, 32, 16]] },  # Expanded filter options
        'activation': {'values': ['relu', 'gelu', 'silu', 'mish']},
        'batch_norm': {'values': [True, False]},
        'dropout_rate': {'values': [0.2, 0.3]},
        'data_augmentation': {'values': [True, False]},
        'filter_organization': {'values': ['same', 'double', 'half']},
        'kernel_size': {'values': [[3,3,3,3,3], [3,3,5,3,3], [3,5,3,5,3], [5,5,5,5,5], [5,7,7,3,5]]},  # New parameter added
        'dense_neurons': {'values':[[512], [256], [128], [64,128,256], [512, 256, 64]]}
    }
}


In [ ]:
# Initialize sweep and run
sweep_id = wandb.sweep(sweep_config, project='DL_A2')
wandb.agent(sweep_id, train_model, count=100)  # Run 20 experiments


Create sweep with ID: liu6yrfd
Sweep URL: https://wandb.ai/ma23m011-iit-madras/DL_A2/sweeps/liu6yrfd


wandb: Agent Starting Run: ynumgry5 with config:
wandb: 	activation: silu
wandb: 	batch_norm: True
wandb: 	data_augmentation: False
wandb: 	dense_neurons: [128]
wandb: 	dropout_rate: 0.3
wandb: 	filter_organization: double
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	num_filters: [64, 64, 64, 64, 64]


train_accuracy,▁▃▄▅▆▆▇▇██
train_loss,█▆▅▄▄▃▂▂▁▁
val_accuracy,▁▁▅▅▅▆███▇
val_loss,█▇▄▄▄▂▂▁▁▃
train_accuracy,39.12989
train_loss,1.73527
val_accuracy,36.25
val_loss,1.84792


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: przuasxn with config:
wandb: 	activation: silu
wandb: 	batch_norm: True
wandb: 	data_augmentation: False
wandb: 	dense_neurons: [256]
wandb: 	dropout_rate: 0.2
wandb: 	filter_organization: same
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	num_filters: [32, 32, 32, 32, 32]


train_accuracy,▁▃▄▅▆▆▇▇▇█
train_loss,█▆▅▄▄▃▃▂▂▁
val_accuracy,▁▃▄▅▃▆▅▆█▇
val_loss,█▆▅▄▄▂▃▃▁▂
train_accuracy,41.03013
train_loss,1.66516
val_accuracy,38.15
val_loss,1.78383


...

wandb: Agent Starting Run: o0bu903x with config:
wandb: 	activation: gelu
wandb: 	batch_norm: True
wandb: 	data_augmentation: False
wandb: 	dense_neurons: [64, 128, 256]
wandb: 	dropout_rate: 0.3
wandb: 	filter_organization: half
wandb: 	kernel_size: [5, 7, 7, 3, 5]
wandb: 	num_filters: [16, 32, 64, 128, 256]


train_accuracy,▁▃▄▅▆▆▆▇██
train_loss,█▆▅▄▃▃▂▂▂▁
val_accuracy,▁▂▅▆████▆█
val_loss,█▇▅▃▃▂▁▁▂▁
train_accuracy,28.016
train_loss,1.9825
val_accuracy,29.05
val_loss,1.96304


wandb: Agent Starting Run: w2awgwoa with config:
wandb: 	activation: silu
wandb: 	batch_norm: False
wandb: 	data_augmentation: True
wandb: 	dense_neurons: [256]
wandb: 	dropout_rate: 0.3
wandb: 	filter_organization: half
wandb: 	kernel_size: [3, 5, 3, 5, 3]
wandb: 	num_filters: [32, 32, 32, 32, 32]


train_accuracy,▁▄▅▆▆▇▇▇██
train_loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▃▄▅▆▆▆▇██
val_loss,█▆▄▄▃▂▂▂▁▁
train_accuracy,36.72959
train_loss,1.786
val_accuracy,37.65
val_loss,1.78162


wandb: Agent Starting Run: z7jar3f8 with config:
wandb: 	activation: mish
wandb: 	batch_norm: True
wandb: 	data_augmentation: True
wandb: 	dense_neurons: [256]
wandb: 	dropout_rate: 0.3
wandb: 	filter_organization: half
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	num_filters: [64, 64, 64, 64, 64]
